In [1]:
import redback 
from redback.transient_models.afterglow_models import tophat
from redback.simulate_transients import SimulateOpticalTransient 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.lines as mlines
from matplotlib.lines import Line2D
from astropy.cosmology import Planck18 as cosmo
from matplotlib import rcParams

# Disable LaTeX rendering
rcParams['text.usetex'] = False
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']  # Use Arial or any other available sans-serif font

import scipy.optimize as scipy
from scipy import stats
import math 
import bilby
from bilby.core.prior import Uniform

from inspect import isfunction
from redback.utils import logger, citation_wrapper, calc_ABmag_from_flux_density, lambda_to_nu
from redback.constants import day_to_s, speed_of_light, solar_mass, proton_mass, electron_mass, sigma_T
from redback.sed import get_correct_output_format_from_spectra
import astropy.units as uu
import numpy as np
from collections import namedtuple
from scipy.special import erf
from scipy.interpolate import interp1d
import afterglowpy as afterglow


No module named 'lalsimulation'
lalsimulation is not installed. Some EOS based models will not work. Please use bilby eos or pass your own EOS generation class to the model
12:15 bilby INFO    : Running bilby version: 2.3.0
12:15 redback INFO    : Running redback version: 1.0.2


In [4]:
model = 'nested tophat'
name = 'GRB221009A'
redshift = 1.151

#read in data from csv files for each band
#figure out x ray columns ???
#x_ray_data = pd.read_csv('/Users/helenagrabham/SUMMER INTERNSHIP/filters data/XRT_10keV_191223.txt', sep = '\t', header = None, names = ['time', 'flux', 'flux_err', 'frequency'])
radio_data = pd.read_csv('/Users/helenagrabham/SUMMER INTERNSHIP/filters data/radio_data_2.csv')  
radio_data_2 = pd.read_csv('/Users/helenagrabham/SUMMER INTERNSHIP/filters data/newradio_data.csv')

#discrepancy between uJy and mJy units in radio data uJy x10^3 = mJy
time= radio_data['Midtime '].values
flux_density = radio_data['Flux Density uJy'].values/1e3
flux_density_err = radio_data['Flux Density Error uJy'].values /1e3
frequency = radio_data['Frequency Hz'].values


time = np.concatenate((time, radio_data_2[' time-t0'].values))
flux_density = np.concatenate((flux_density, radio_data_2[' Flux density (mJy)'].values))
flux_density_err = np.concatenate((flux_density_err, radio_data_2[' err (mJy)'].values))
frequency = np.concatenate((frequency, radio_data_2['Frequency Hz'].values))


print(f"Time : {len(time)}")
print(f"Frequency : {len(frequency)}")
print(f"Flux Density : {len(flux_density)}")
print(f"Flux Density Error : {len(flux_density_err)}")


afterglow = redback.transient.Afterglow(name=name, data_mode = 'flux_density', time = time, 
                                        flux_density = flux_density, flux_density_err = flux_density_err, 
                                        frequency = frequency,  redshift=redshift, model=model)


# Define the model
def nested_tophat(time, nu, p, **kwargs):
    return afterglow.flux_density(time, nu, p, **kwargs)

model = nested_tophat
name = 'GRB221009A'
redshift = 1.151

""" #Define parameters 
thc = 

# Define the priors
priors = dict()
priors['p'] = Uniform(0, 1e-3, name='p')
priors['g0'] =
priors['logepse'] =
priors['logepsb'] = 
priors['xiN'] =  """


""" afterglow.plot_data 

result = redback.fit_model(model=model, sampler='dynesty', nlive=200, transient=afterglow,
                           prior=priors, sample='rslice', resume=False)

result.plot_lightcurve(model=nested_tophat, time=afterglow.time, frequency=afterglow.frequency,  p=result.median['p'], redshift=redshift, name=name, save=True, show=True) """

12:15 redback INFO    : Metadata does not exist for this event.
12:15 redback INFO    : Setting metadata to None. This is not an error, but a warning that no metadata could be found online.


Time : 300
Frequency : 300
Flux Density : 300
Flux Density Error : 300


" afterglow.plot_data \n\nresult = redback.fit_model(model=model, sampler='dynesty', nlive=200, transient=afterglow,\n                           prior=priors, sample='rslice', resume=False)\n\nresult.plot_lightcurve(model=nested_tophat, time=afterglow.time, frequency=afterglow.frequency,  p=result.median['p'], redshift=redshift, name=name, save=True, show=True) "

In [6]:

x_ray_data = pd.read_csv('/Users/helenagrabham/SUMMER INTERNSHIP/filters data/XRT_10keV_191223.txt',
                          sep = '\t', header = None , names = ['time', 't_err', '-t_err', 'flux', 'flux_err', '-flux_err'])
#print(x_ray_data.head(3))
# Add the new frequency column
x_ray_data['Frequency'] = 2e18

print("Data with new frequency column:")
      
output = '/Users/helenagrabham/SUMMER INTERNSHIP/filters data/XRT_10keV_191223.csv'
x_ray_data.to_csv(output, index = False, header = True)

#x_ray_data.columns = ['time', 't_err', '-t_err', 'flux', 'flux_err', '-flux_err']
print(x_ray_data.head(3))

csv_data = pd.read_csv(output)
print(len(csv_data['time'].values))
print(len(csv_data['flux'].values))
print(len(csv_data['flux_err'].values))
print(len(csv_data['Frequency'].values))
#extract variables from larger dataframes and add to numpy arrays for individual variables 
#concatenate - two arrays 
#append - array + item 
time = np.append(time, csv_data['time'].values)
flux_density = np.append(flux_density, csv_data['flux'].values)
flux_density_err = np.append(flux_density_err, csv_data['flux_err'].values)
frequency = np.append(frequency, csv_data['Frequency'].values)

print(f"Time : {len(time)}")
print(f"Frequency : {len(frequency)}")
print(f"Flux Density : {len(flux_density)}")
print(f"Flux Density Error : {len(flux_density_err)}")

Data with new frequency column:
       time  t_err  -t_err      flux  flux_err  -flux_err     Frequency
0  3376.254  2.759  -2.846  0.001537  0.000065  -0.000065  2.000000e+18
1  3381.482  2.355  -2.470  0.001462  0.000060  -0.000060  2.000000e+18
2  3386.421  2.593  -2.584  0.001781  0.000073  -0.000073  2.000000e+18
1603
1603
1603
1603
Time : 3506
Frequency : 3506
Flux Density : 3506
Flux Density Error : 3506


In [ ]:
afterglow.plot_data()

# You need to create your own priors for this new model.
# The model has two parameters l0 and alpha. We use bilby priors for this
priors = bilby.core.prior.PriorDict()


# Call redback.fit_model to run the sampler and obtain GRB result object
result = redback.fit_model(model=model, sampler='dynesty', nlive=200, transient=afterglow,
                           prior=priors, sample='rslice', resume=False)


In [ ]:
#uJy ---> mJy
# 0.001 --> 0.000001 
#divide not multiply 

#column for frequency 2e18 

#time, error on time, flux density, error on fklux density, frequency column with all one value 